In [1]:
using BenchmarkTools

function algo1(n)
    return sum(1:n)
end

function algo2(n)
    return n * (n + 1) ÷ 2
end

@btime algo1(10^6)
@btime algo2(10^6)


  1.200 ns (0 allocations: 0 bytes)
  1.200 ns (0 allocations: 0 bytes)


500000500000

In [2]:
using BenchmarkTools, Plots

function benchmark_scaling(f, sizes)
    times = Float64[]
    for n in sizes
        t = @belapsed f($n)   # high-precision elapsed time
        push!(times, t)
    end
    return times
end

sizes = 10 .^ (2:6)
times1 = benchmark_scaling(algo1, sizes)
times2 = benchmark_scaling(algo2, sizes)

plot(sizes, times1, label="algo1", xscale=:log10, yscale=:log10, xlabel="Input size n", ylabel="Time (s)")
plot!(sizes, times2, label="algo2")


UndefVarError: UndefVarError: `f` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [3]:
Profile.clear()
@profile algo1(10^6)
using ProfileView   # nice GUI flamegraphs
ProfileView.view()


UndefVarError: UndefVarError: `Profile` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: Profile is loaded but not imported in the active module Main.

In [4]:
using PAPI

PAPI.start_counters(["PAPI_FP_OPS"]) do
    A = rand(1000,1000)
    B = rand(1000,1000)
    C = A * B
end


InitError: InitError: UndefVarError: `libpapi` not defined in `PAPI`
Suggestion: check for spelling errors or missing imports.
during initialization of module PAPI

In [5]:
using IterativeSolvers
using LinearAlgebra

# Example system: A * x = b
A = [4.0 1.0;
     1.0 3.0]
b = [1.0, 2.0]

# Solve with GMRES
x, history = gmres(A, b; log=true)

println("Solution x = ", x)
println("Residual norms = ", history.data)

# exact solution
x_exact = A \ b
println("Exact solution x = ", x_exact)

Solution x = [0.09090909090909091, 0.6363636363636362]
Residual norms = Dict{Symbol, Any}(:reltol => 1.4901161193847656e-8, :abstol => 0.0, :resnorm => [0.5423261445466404, 1.0092936587501423e-16])
Exact solution x = [0.09090909090909091, 0.6363636363636364]
